In [1]:
import cv2 as cv
import sys
import numpy as np
import matplotlib.pyplot as plt #used it to find the coordinates of region of interest by plotting a single frame from video

In [2]:
def canny(image):
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)#converts coloured image to greyscale
    blur = cv.GaussianBlur(gray,(5,5),0)#blurs the image to remove noise
    #parameters include the source image, kernel size and standard deviation applied in x direction
    canny = cv.Canny(blur, 50,150)#detects edges in image which satisfy the threshold values, here 50 and 150 are min and max threshold respectively
    return canny

In [3]:
def average_slope_intercept(image, lines): 
    left_fit = [] #creating empty list for storing parameters of left predicted path
    right_fit = [] #creating empty list for storing parameters of right predicted path
    for line in lines: 
        x1, y1, x2, y2 = line.reshape(4) #storing x and y coordinates in respective variables
          
         
        parameters = np.polyfit((x1, x2), (y1, y2), 1) #extracts the slope and intercept parameters of the lines by fitting a 1 degree polynomial in the given points 
        slope = parameters[0] 
        intercept = parameters[1] 
        if slope < 0: 
            left_fit.append((slope, intercept)) #for lines on the left path slope is negative 
        else: 
            right_fit.append((slope, intercept)) #for lines on the right path slope is positive
              #this is due to taking positive y axis downward
    left_fit_average = np.average(left_fit, axis = 0) #averaging the values
    right_fit_average = np.average(right_fit, axis = 0) 
    left_line = make_coordinates(image, left_fit_average) 
    right_line = make_coordinates(image, right_fit_average) 
    return np.array([left_line, right_line])

In [4]:
def make_coordinates(image, line_parameters):
    slope,intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])#making an array of the respective points


In [5]:
def region_of_interest(image):
    height = image.shape[0] #height becomes the entire row of the image 
    polygon = np.array([[(200,height),(1150,height),(550,250)]]) #creates a polygon array of the defined points
    mask = np.zeros_like(image) #creating a completely black image, same size as the image
    cv.fillPoly(mask,polygon,255) #fills the black mask with white color in the region defined by polygon array points
    masked_image = cv.bitwise_and(image,mask) #bitwise_and isolates the defined region and makes the surrounding region black
    return masked_image

In [6]:
def display_lines(image, lines): 
    line_image = np.zeros_like(image) #creating a completely black image, same size as the image
    if lines is not None: 
        for x1, y1, x2, y2 in lines: 
            cv.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10) #creates a line joining (x1,y1) and (x2,y2) of blue color and thickness 10px
                                                                    
    return line_image 

In [7]:
cap = cv.VideoCapture("/home/aarush/Downloads/challenge_video.mp4") #creating video capture object

fourcc = cv.VideoWriter_fourcc(*'XVID') #defining the codec
out = cv.VideoWriter('Output_Video.mp4',fourcc,20,(640,480))

while(cap.isOpened()): 
    _, frame = cap.read() #reading the video frames
    canny_image = canny(frame)

    cropped_image = region_of_interest(canny_image)

    lines = cv.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]),minLineLength =40,maxLineGap =5)#predicts the edges and joins them by creating lines
    #parameters include source image, 
    #2 is resolution paramter rho in px. More px means less accuracy
    #np.pi/180 is resolution parameter theta in radians
    #np.array([]) stores the coordinates of the start and end points of the lines
    #minLineLength defines the minimum pixels to consider in a line to be considered valid
    #maxLineGap defines the max pixels between two lines so that they may merge to become a single line.Above the max value they are considered two different lines
    averaged_lines = average_slope_intercept(frame,lines)
    
    line_image = display_lines(frame, averaged_lines)
    
    combo_image = cv.addWeighted(frame, 0.8, line_image,1,0) #blending the frames and lines together 
    #giving a weight of 0.8 to frames and 1.0 to line_image, so that they are easily visible
    
    out.write(frame)#saving the video frames
    
    cv.imshow("image",combo_image)#displaying the video frames
    if cv.waitKey(1) & 0xFF == ord('q'):       
        break

cap.release()  #releasing cap object
cv.destroyAllWindows()


/home/aarush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RankWarning: Polyfit may be poorly conditioned
/home/aarush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RankWarning: Polyfit may be poorly conditioned
/home/aarush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RankWarning: Polyfit may be poorly conditioned
/home/aarush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RankWarning: Polyfit may be poorly conditioned
/home/aarush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RankWarning: Polyfit may be poorly conditioned
/home/aarush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RankWarning: Polyfit may be poorly conditioned
/home/aarush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RankWarning: Polyfit may be poorly conditioned
/home/aarush/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: RankWarning: Polyfit may be poorly conditioned
/home/aarush/.local/lib/python3.

OverflowError: signed integer is greater than maximum

In [ ]:
# import cv2 as cv
# import sys
# import numpy as np
# import matplotlib.pyplot as plt 


In [ ]:
# def canny(image):
#     gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)#converts coloured image to greyscale
#     blur = cv.GaussianBlur(gray,(5,5),0)#blurs the image to remove noise 
#     canny = cv.Canny(blur, 50,150)#detects edges in image which satisfy the threshold values
#     return canny
# image = cv.imread('/home/aarush/Downloads/Screenshot from 2021-04-10 12-28-36.jpg')
# new =canny(image)
# plt.imshow(new)
# plt.imshow(new)
# plt.show()